<b>Data mining project - 2020/21</b><br>
<b>Author</b>: [Alexandra Bradan](https://github.com/alexandrabradan)<br>
<b>Python version</b>: 3.x<br>
<b>Last update: 07/01/2021<b>

In [44]:
%matplotlib inline

# general libraries
import sys
import math
import operator
import itertools
import pydotplus
import collections
import missingno as msno
from pylab import MaxNLocator
from collections import Counter
from collections import defaultdict
from IPython.display import Image

# pandas libraries
import pandas as pd
from pandas import DataFrame
from pandas.testing import assert_frame_equal

# visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

# numpy libraries
import numpy as np
from numpy import std
from numpy import mean
from numpy import arange
from numpy import unique
from numpy import argmax
from numpy import percentile

# scipy libraries
import scipy.stats as stats
from scipy.stats import kstest
from scipy.stats import normaltest

# sklearn libraries
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer  # explicitly require this experimental feature
from sklearn.impute import IterativeImputer

from sklearn import tree
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.pipeline import make_pipeline as imbmake_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, fbeta_score, recall_score, precision_score, classification_report, roc_auc_score 

In [45]:
data_directory = "../../../data/"
plot_directory = "../../../plots/DataUnderstanding/"
TR_cleaned_file = data_directory + "Discretized_One_Hot_Encoding_Train_HR_Employee_Attrition.csv"
TS_file = data_directory + "Discretized_One_Hot_Encoding_Test_HR_Employee_Attrition.csv"

In [46]:
df_cleaned = pd.read_csv(TR_cleaned_file, sep=",") 

In [47]:
df_ts = pd.read_csv(TS_file , sep=",") 

In [48]:
df_cleaned.shape

(883, 102)

In [49]:
df_ts.shape

(219, 98)

<h2> Split dataset in Training and Test set </h2>

In [50]:
for column_name in df_cleaned.columns:
    if "OverallSatisfaction" in column_name:
        del df_cleaned[column_name]
df_cleaned.shape

(883, 95)

In [51]:
for column_name in df_ts.columns:
    if "OverallSatisfaction" in column_name:
        del df_ts[column_name]
df_ts.shape

(219, 92)

In [52]:
y = df_cleaned['Attrition']
df1 = df_cleaned.copy()
X = df1.drop('Attrition', axis=1)
print(X.shape)

(883, 94)


In [53]:
# summarize dataset
classes = unique(y)
total = len(y)
for c in classes:
    n_examples = len(y[y==c])
    percent = n_examples / total * 100
    print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))

> Class=0 : 730/883 (82.7%)
> Class=1 : 153/883 (17.3%)


<h1> Imbalanced target variable </h1>

Since our training set is highly unbalanced with respect to the target attribute object of the classification (Attrition=Yes: 153 (17.33%), Attrition=No:730 (82.67%)), we try to solve this problem using three resampling techniques: 

- **StratifiedKFold**: split a dataset randomly, in such a way that maintains the same class distribution in each fold.
- **oversampling**: duplicates examples from minority class;  ==> may lead to overfitting
- **undersampling**: deletes or filters examples from the majority class.  ==> may lead to underfitting

In [55]:
def plot_confusion_matrix(cm, classes, normalize, title, cmap):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    # plt.clf()

<h2> Stratified 70-30 holdout </h2>

In [56]:
"""X_filtered = X[best_absolute_features]
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y, test_size=0.3, random_state=1, stratify=y)"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

In [57]:
n_splits = 3  # StratifiedKFold splits
no_skill = len(y[y==1]) / len(y)  # PR_curve random model AUC
sampling_methods_info = {}

In [58]:
def report(results, n_top, scoring, scorings):
    configurations = {}
    c_i = 0
    for i in range(1, n_top + 1):
        # retrieeve best i-th model's score index
        candidates = np.flatnonzero(results['rank_test_score'] == i)  # returns array([score_index])
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean training score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_train_score'][candidate],
                  results['std_train_score'][candidate]))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            configurations[c_i] = results['params'][candidate]
            c_i += 1 
    return configurations


def report_multiple(results, n_top, scoring, scorings):
    configurations = {}
    c_i = 0
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_' + str(scoring)] == i)
        for candidate in candidates:
            """print("Model with rank: {0}".format(i))
            print("Mean training score:", end = '')
            for s in scorings:
                print( "   " + s + ": {0:.3f} (std: {1:.3f})".format(
                      results['mean_train_' + s][candidate],
                      results['std_train_' + s][candidate]), end = '')
            

            print("Mean validation score:", end = '')
            for s in scorings:
                print( "   " + s + ": {0:.3f} (std: {1:.3f})".format(
                      results['mean_test_' + s][candidate],
                      results['std_test_' + s][candidate]), end = '')

            print("Parameters: {0}".format(results['params'][candidate]))
            print("")"""
            configurations[c_i] = results['params'][candidate]
            c_i += 1 
    return configurations

In [59]:
def update_best_model_info(resampling_label, models_u, roc_auc_models_u_val, precision_recall_auc_models_u_val,
                           best_model_index_u, best_ap_model_index_u, models_thresh, best_thresh, 
                           ap_best_thresh, scoring, refit):
    # keep_track_of_best_model_info
    sampling_methods_info[resampling_label] = {}
    cnf = models_u[best_model_index_u]
    sampling_methods_info[resampling_label]["model_param"] = {}
    sampling_methods_info[resampling_label]["model_param"]["criterion"] = cnf.criterion
    sampling_methods_info[resampling_label]["model_param"]["max_features"] = cnf.max_features
    sampling_methods_info[resampling_label]["model_param"]["max_depth"] = cnf.max_depth
    sampling_methods_info[resampling_label]["model_param"]["min_samples_split"] = cnf.min_samples_split
    sampling_methods_info[resampling_label]["model_param"]["min_samples_leaf"] = cnf.min_samples_leaf
    sampling_methods_info[resampling_label]["model_param"]["class_weight"] = cnf.class_weight
    sampling_methods_info[resampling_label]["roc_auc_best_model_index"] = best_model_index_u
    sampling_methods_info[resampling_label]["roc_auc"] = roc_auc_models_u_val
    sampling_methods_info[resampling_label]["ap_best_model_index"] = best_ap_model_index_u
    sampling_methods_info[resampling_label]["ap"] = precision_recall_auc_models_u_val
    sampling_methods_info[resampling_label]["model_threshold"] = models_thresh
    sampling_methods_info[resampling_label]["roc_auc_threshold"] = best_thresh
    sampling_methods_info[resampling_label]["pr_ap_threshold"] = ap_best_thresh
    sampling_methods_info[resampling_label]["scoring"] = scoring
    sampling_methods_info[resampling_label]["refit"] = refit

In [60]:
def get_decision_tree_classifier_params(sampler, resampling_label):
    param_list = {'criterion': ['gini', 'entropy'],
                  'max_features': [None, 'auto', 'sqrt', 'log2'],
                  'max_depth': [None] + list(np.arange(1, len(X.columns) + 1)),
                  # 'min_samples_split': [0.1, 0.01, 0.001], # ceil(min_samples_split * n_samples)
                  'min_samples_split':[2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                  # 'min_samples_leaf': [0.05, 0.005, 0.0005], # ceil(min_samples_split * n_samples)
                  'min_samples_leaf':[2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                  'class_weight': ['balanced', [{0:1,1:10}, {0:1,1:100}, {0:1,1:1000}],
                                   None],  # balanced={0:153,1:883}, None={0:1,1:1} (Cost-sensitive) 
                 }
    new_params = {'decisiontreeclassifier__' + key: param_list[key] for key in param_list}
    return sampler, new_params

In [61]:
def to_labels(pos_probs, threshold):
    # apply threshold to positive probabilities to create labels
    return (pos_probs >= threshold).astype('int')

In [62]:
def get_model_thresholds(model):
    model.fit(X_train, y_train)
    # predict probabilities
    yhat = model.predict_proba(X_test)
    # keep probabilities for the positive outcome only
    probs = yhat[:, 1]
    # define thresholds
    thresholds = arange(0, 1, 0.001)
    # evaluate each threshold
    scores = [f1_score(y_test, to_labels(probs, t)) for t in thresholds]
    # get best threshold
    ix = argmax(scores)
    print( 'ModelThreshold=%.3f, F-measure=%.5f ' % (thresholds[ix], scores[ix]))
    return thresholds[ix]

In [63]:
def my_grid_search(sampler, resampling_label, scoring, refit):    

    # declare K-Fold validation to use
    # declare model to use
    # initialize sampler and GridSearchCV's hyperparameters to tune
    cv = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
    model = DecisionTreeClassifier(random_state=42)  #DECISION TREE
    sampler, new_params = get_decision_tree_classifier_params(sampler, resampling_label)
    
    # declare a imbalance Pipeline to use 
    # (I'm using imblearn.pipeline since it allows to performe both K-Fold (resampling only on TR's folds)
    # and model's tuning, at same time)
    if sampler != "":
        imba_pipeline = imbmake_pipeline(sampler, model)
    imba_pipeline = imbmake_pipeline(model)

    # perform GridSearchCV (hyperparameters tuning)
    grid_search = GridSearchCV(imba_pipeline, param_grid=new_params, cv=cv, scoring=scoring, 
                                 refit=refit, n_jobs=-1, verbose=1, return_train_score=True)
    # build inductive model on TR
    grid_search.fit(X_train, y_train)
    results = grid_search.cv_results_
    cnfs = report_multiple(results, n_top=3, scoring=refit, scorings=scoring)
    
    # train_models:
    if sampler != "":
        x_u_train_resampled, y_u_train_resampled = sampler.fit_resample(X_train, y_train)
    else:
        x_u_train_resampled, y_u_train_resampled = X_train, y_train
        
    models_u = []
    y_pred_vals_u = []
    y_pred_trains_u = []
    hyper_ps = grid_search.cv_results_
    model_index = 0
    for cnf in cnfs.values():
        criterion = cnf['decisiontreeclassifier__criterion']
        max_features = cnf['decisiontreeclassifier__max_features']
        max_depth = cnf['decisiontreeclassifier__max_depth']
        min_samples_split = cnf['decisiontreeclassifier__min_samples_split']
        min_samples_leaf = cnf['decisiontreeclassifier__min_samples_leaf']
        class_weight = cnf['decisiontreeclassifier__class_weight']
        clf = DecisionTreeClassifier(criterion=criterion, 
                                     max_features=max_features,
                                     max_depth=max_depth,
                                     min_samples_split=min_samples_split, 
                                     min_samples_leaf=min_samples_leaf,
                                     class_weight=class_weight)
        clf = clf.fit(x_u_train_resampled, y_u_train_resampled)
        models_u.append(clf)
        y_pred = clf.predict(X_test)
        y_pred_tr = clf.predict(x_u_train_resampled)
        y_pred_vals_u.append(y_pred)
        y_pred_trains_u.append(y_pred_tr)

        # plot current model's features importance
        x = []
        y = []
        for i in range(len(clf.feature_importances_)):
            if clf.feature_importances_[i] > 0:
                x.append(list(X.columns)[i])
                y.append(clf.feature_importances_[i])
        plt.bar(x, y)
        plt.xticks(rotation=90)
        plt.ylabel("Score")
        plt.xlabel("Features")
        plt.title("Features importance for model %d" % (model_index))
        plt.show()
        model_index += 1


    # test_models
    roc_auc_models_u_val = []  # models' TS ROC_AUC 
    precision_recall_auc_models_u_val = []  # models' TS PRECISION_RECALL_AUC
    for i in range(0, len(cnfs)):
        fpr, tpr, thresholds = roc_curve(y_u_train_resampled, y_pred_trains_u[i])
        roc_auc = auc(fpr, tpr)
        roc_auc = roc_auc_score(y_u_train_resampled, y_pred_trains_u[i], average="weighted")
        print("model {}".format(i))
        print('Train Accuracy %s' % accuracy_score(y_u_train_resampled, y_pred_trains_u[i]))
        print('Train Precision %s' % precision_score(y_u_train_resampled, y_pred_trains_u[i], average="weighted"))
        print('Train Recall %s' % recall_score(y_u_train_resampled, y_pred_trains_u[i], average="weighted"))
        print('Train F1-score %s' % f1_score(y_u_train_resampled, y_pred_trains_u[i], average="weighted"))
        print('Train F2-score %s' % fbeta_score(y_u_train_resampled, y_pred_trains_u[i], average="weighted",
                                               beta=2))
        print("Train roc_auc: {}".format(roc_auc))
        cm = confusion_matrix(y_u_train_resampled, y_pred_trains_u[i])
        plot_confusion_matrix(cm, models_u[i].classes_, False, "TR's confusion matrix for model %d" %\
                              (i), plt.cm.Blues)
        plt.show()

        roc_auc = roc_auc_score(y_test, y_pred_vals_u[i], average="weighted")
        roc_auc_models_u_val.append(roc_auc)
        print("Validation roc_auc: {}".format(roc_auc))
        
        pr_ap = average_precision_score(y_test, y_pred_vals_u[i], average="weighted")
        precision_recall_auc_models_u_val.append(pr_ap)
        print("Validation precision_recall_ap: {}".format(pr_ap))
        
        print('\nValidation Accuracy %s' % accuracy_score(y_test, y_pred_vals_u[i]))
        print('Validation Precision %s' % precision_score(y_test, y_pred_vals_u[i], average="weighted"))
        print('Validation Recall %s' % recall_score(y_test, y_pred_vals_u[i], average="weighted"))
        print('Validation F1-score %s' % f1_score(y_test, y_pred_vals_u[i], average="weighted"))
        print('Validation F2-score %s' % fbeta_score(y_test, y_pred_vals_u[i], average="weighted", beta=2))
        print(classification_report(y_test, y_pred_vals_u[i]))
        cm = confusion_matrix(y_test, y_pred_vals_u[i])
        plot_confusion_matrix(cm, models_u[i].classes_, False, "VS' confusion matrix for model %d" %\
                              (i), plt.cm.Blues)
        plt.show()
        print('Parameters model %s and resampling_label=%s: criterion=%s, max_features=%s, max_depth=%s, min_samples_split=%s, min_samples_leaf=%s, class_weight=%s"' % 
              (i, resampling_label, models_u[i].criterion, models_u[i].max_features, models_u[i].max_depth, 
               models_u[i].min_samples_split, models_u[i].min_samples_leaf, models_u[i].class_weight))
    
    # get_best_model_index based on Precision_Recall_AP
    for i in range(0,len(cnfs)):
        print("model {} - precision_recall_ap: {}".format(i, precision_recall_auc_models_u_val[i]))
    best_pr_ap = max(precision_recall_auc_models_u_val)
    best_ap_model_index_u = precision_recall_auc_models_u_val.index(best_pr_ap)
    print("\nBEST MODEL INDEX = %d" % best_ap_model_index_u)

        
    # get_best_model_index based on ROC_AUC
    for i in range(0,len(cnfs)):
        print("model {} - roc_auc: {}".format(i, roc_auc_models_u_val[i]))
    best_roc_auc = max(roc_auc_models_u_val)
    best_model_index_u = roc_auc_models_u_val.index(best_roc_auc)
    print("\nBEST MODEL INDEX = %d" % best_model_index_u)
    
    models_thresh = get_model_thresholds(models_u[best_model_index_u])
        
    # draw_best_model_precision_recall_curve
    plt.figure(figsize=(8, 5))
    precision, recall, ap_thresholds = precision_recall_curve(y_test, y_pred_vals_u[best_model_index_u])
    # convert to f-measure
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f-measure
    ix = argmax(fscore)
    print( ' Best Threshold=%f, F-measure=%.3f ' % (ap_thresholds[ix], fscore[ix]))
    ap_best_thresh = ap_thresholds[ix]
    plt.plot(precision, recall, label='PR curve (AP=%0.4f)' %\
             (precision_recall_auc_models_u_val[best_model_index_u]))
        
    # calculate the no skill line as the proportion of the positive class
    # plot the no skill precision-recall curve
    plt.plot([0, 1], [no_skill, no_skill], 'k--', color="red", label='Random model') 
    plt.xlim([0.0, 1])
    plt.ylim([0.0, 1])
    plt.xlabel('Recall')
    plt.ylabel('Precision') 
    plt.tick_params(axis='both', which='major')
    plt.legend(loc="upper right", fontsize=14, frameon=False)
    plt.title("Model %d's PR AP(average precision)" % best_model_index_u)
    plt.show()


    # draw_best_model_roc_auc
    plt.figure(figsize=(8, 5))
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_vals_u[best_model_index_u])
    # Youden’s J statistic 
    J = tpr - fpr
    # locate best threshold's index
    ix = argmax(J)
    best_thresh = thresholds[ix]
    print( ' Best Threshold=%f ' % (best_thresh))
    plt.plot(fpr, tpr, label='ROC curve (AUC=%0.4f)' % (roc_auc_models_u_val[best_model_index_u]))
        
    plt.plot([0, 1], [0, 1], 'k--', color="red", label='Random model') 
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate') 
    plt.tick_params(axis='both', which='major')
    plt.legend(loc="lower right", fontsize=14, frameon=False)
    plt.title("Model %d's ROC AUC" % best_model_index_u)
    plt.show()
    
    # keap track of best model's hyperparameters
    update_best_model_info(resampling_label, models_u, roc_auc_models_u_val, precision_recall_auc_models_u_val,\
                          best_model_index_u, best_ap_model_index_u,  models_thresh, best_thresh, \
                          ap_best_thresh, scoring, refit)
    
    return models_u, roc_auc_models_u_val, precision_recall_auc_models_u_val, best_model_index_u,\
                                                                                    best_ap_model_index_u

<h2>Plain TR</h2>

In [64]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

In [65]:
"""scoring_base = ["recall"]
scoring_pool = ["f1", "roc_auc", "precision", "f1_weighted", "average_precision", "roc_auc_ovo_weighted"] 
refit_pool = ["recall", "f1", "roc_auc", "f1_weighted", "average_precision", "roc_auc_ovo_weighted"]"""

'scoring_base = ["recall"]\nscoring_pool = ["f1", "roc_auc", "precision", "f1_weighted", "average_precision", "roc_auc_ovo_weighted"] \nrefit_pool = ["recall", "f1", "roc_auc", "f1_weighted", "average_precision", "roc_auc_ovo_weighted"]'

In [66]:
scoring_base = ["recall"]
scoring_pool = ["roc_auc", "precision", "f1", "average_precision"] 
refit_pool = ["recall"]

In [67]:
scoring_sets = set()
for s in scoring_pool:
    # scoring = scoring_base + list(powerset(scoring_pool))
    tmp_list = list(powerset(scoring_pool))
    max_set_size = len(scoring_pool)
    for tmp_tuple in tmp_list:
        tmp_tuple += (scoring_base[0],) 
        scoring_sets.add(tmp_tuple)

In [ ]:
for scoring in scoring_sets:
    scoring = list(scoring) 
    for refit in refit_pool:
        resampling_label = "scoring=" + str(scoring) + "refit=" + str(refit)
        print(resampling_label)
        models_u, roc_auc_models_u_val, precision_recall_auc_models_u_val, best_model_index_u, best_ap_model_index_u  = \
                                                        my_grid_search("", resampling_label, scoring, "recall")
        break

In [ ]:
max_roc_auc_key = ""
max_roc_auc_mode_index = -1
max_roc_auc_value = -1

max_ap_key = ""
max_ap_model_index = -1
max_ap = -1
for key, value, in sampling_methods_info.items():
    roc_auc_best_model_index = sampling_methods_info[key]["roc_auc_best_model_index"]
    if sampling_methods_info[key]["roc_auc"][roc_auc_best_model_index] >= max_roc_auc_value:
        max_roc_auc_value = sampling_methods_info[key]["roc_auc"][roc_auc_best_model_index]
        max_roc_auc_mode_index = roc_auc_best_model_index
        max_roc_auc_key = key
        
    ap_best_model_index = sampling_methods_info[key]["ap_best_model_index"]
    if sampling_methods_info[key]["ap"][ap_best_model_index] >= max_ap:
        max_ap = sampling_methods_info[key]["ap"][ap_best_model_index]
        max_ap_model_index = ap_best_model_index
        max_ap_key = key
        
print("max_roc_auc_key", max_roc_auc_key)
print("max_roc_auc_mode_index", max_roc_auc_mode_index)
print("max_roc_auc", max_roc_auc_value)
print()
print("max_ap_model_index", max_ap_model_index)
print("max_ap_key", max_ap_key)
print("max_ap", max_ap)